In [10]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
file_path = "E:/CV.pdf"

In [13]:
loader = PyPDFLoader(file_path=file_path)

In [17]:
load = loader.load()

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap= 20
)

splitted_text = text_splitter.split_documents(load)

In [23]:
splitted_text[:1] 

[Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-11-15T23:15:52+00:00', 'author': '', 'keywords': '', 'moddate': '2025-11-15T23:15:52+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'E:/CV.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Muhammad Haris Imtiaz')]

In [24]:
#Embeddings

In [25]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

In [27]:
embed = HuggingFaceEmbeddings(model=embedding_model)

e:\PDF_ChatBot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
final_embed = Chroma.from_documents(
    documents=splitted_text,
    embedding=embed,
    collection_name="pdfs",
    persist_directory="E:/PDF_ChatBot/chroma_langchain_db"
)

In [38]:
len(final_embed)

73

In [ ]:
collection = vector_store._collection
results = collection.get(include=["embeddings", "documents"])

In [47]:
len(results['documents'])

73

In [50]:
results.keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])

#### Now the Functions will be made for the later use  

In [51]:
def load_pdf(file_path):
    loader = PyPDFLoader(file_path=file_path)
    load = loader.load()

    return load

In [65]:
def splitting(load):
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap= 20)

    splitted_text = text_splitter.split_documents(load)

    return splitted_text

In [67]:
def embeds(load):
    embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
    embed = HuggingFaceEmbeddings(model=embedding_model)
    
    final_embed = Chroma.from_documents(
    documents=splitted_text,
    embedding=embed,
    collection_name="pdfs",
    persist_directory="E:/PDF_ChatBot/chroma_langchain_db"
)
    return final_embed
    
    

In [76]:
def custom_embeds(load):
    embedding_model1 = input("Please provide the model for embedding which you want to load: ")
    embedding_model = embedding_model1
    embed = HuggingFaceEmbeddings(model=embedding_model)
    
    final_embed = Chroma.from_documents(
    documents=splitted_text,
    embedding=embed,
    collection_name="pdfs",
    persist_directory="E:/PDF_ChatBot/chroma_langchain_db"
)
    return final_embed
    
    

In [120]:
from dotenv import load_dotenv
import streamlit as st

In [121]:
load_dotenv()

True

In [122]:
if not os.getenv("GOOGLE_API_KEY"):
    st.error("Err")
    st.stop

In [123]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_classic.chains import RetrievalQA

In [124]:
#Setting up the brain

In [125]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3, 
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [126]:
#For retrieval

In [127]:
ret = final_embed.as_retriever(search_kwargs = {"k" : 2})

In [128]:
ret

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001BD1C439E50>, search_kwargs={'k': 2})

In [129]:
#connecting the brain and retrival for the responses

In [130]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = ret
)

In [131]:
qa_chain.invoke("Tell me the experience ")

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (INVALID_ARGUMENT): 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API Key not found. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API Key not found. Please pass a valid API key.'}]}}